In [1]:
import pandas as pd
import numpy as np
from dynGENIE3 import *

In [39]:
lc = pd.read_csv("./path/to/LiveCellData_15cells.csv",index_col=0)
tmm = pd.read_csv("./path/to/TMM_NormlizeFactors.csv",index_col=0)
for idx in lc.columns:
    lc[idx] = lc[idx]*tmm.loc[idx,"norm.factors"]
gn = pd.read_csv('./path/to/GeneID_list.csv',index_col=0)
lc.index = [gn.loc[i,"Approved symbol"] if i in gn.index else i for i in lc.index]

lct=lc.T
filename = "Cell_SamplingTime.csv"
path = "./path/to/"
df = pd.read_csv(path+filename,index_col=0)
new_lct= pd.concat([lct,df], axis=1)

In [28]:
TFs = ['MYC', 'SMAD2', 'TFDP1', 'TFDP2']
CoFs =['EP300','CCNA2','CCNH','CDK1','HDAC1','HDAC2','MAD2L2','MDM2','PLK1','PTTG1','RAD21','RBX1','YWHAB']
Regulators = TFs+CoFs
HV_ccg = ['CDC6', 'CDK1', 'WEE1', 'MYC', 'SMC1A', 'CDK7', 'BUB3', 'SMC3', 'HDAC2',
       'CCNA2', 'CUL1', 'MDM2', 'ANAPC11', 'YWHAE', 'CCNB2', 'MCM3', 'CDKN2A',
       'CHEK2', 'MAD2L1', 'CDC25B', 'YWHAH', 'MCM4', 'RBX1', 'TFDP1', 'CCNH',
       'HDAC1', 'MCM6', 'PTTG1', 'CDC20', 'YWHAZ', 'RAD21', 'SKP2', 'CDC23',
       'GADD45A', 'SKP1', 'YWHAB', 'PRKDC', 'ORC5', 'MCM7', 'CCND3', 'MAD2L2',
       'TFDP2', 'SMAD2', 'CDC7', 'BUB1', 'CDC45', 'ANAPC5', 'ANAPC13', 'SFN',
       'ORC2', 'YWHAQ', 'CCNB1', 'CDK2', 'PCNA', 'EP300', 'PLK1', 'CDKN1A',
       'CCND1']

In [45]:
for cell in range(1,16):
    cell_key = "Cell"+str(cell)
    time = list(new_lct[new_lct["Cell"]==cell_key].sort_values("Time").index)
    detected_whole_ccg = [g for g in HV_ccg if new_lct[new_lct["Cell"]==cell_key][g].sum()>0]
    detected_ccg_reg = list(set(Regulators) & set(detected_whole_ccg))
    print("Num of Reg : "+str(len(detected_ccg_reg)))
    result = dynGENIE3([new_lct.loc[time,detected_whole_ccg].values],
                               [np.array([0, 2.5,5,7.5])],
                               regulators = detected_ccg_reg,
                               gene_names = detected_whole_ccg,nthreads=16)
    GRN_df = pd.DataFrame(result[0],index=detected_whole_ccg,columns=detected_whole_ccg)
    GRN_df.to_csv("./path/to/GRN_Cell_{}_withCoFs.csv".format(cell))

Num of Reg : 16
Tree method: RF
K: sqrt
Number of trees: 1000
alpha min: 0.11435039795955229
alpha max: 7.238763699824235


running jobs on 16 threads
Elapsed time: 3.25 seconds
